**使用multiprocessing实现多核CPU并行处理**  
@Author: Ray  
@Build time: 2022.08.23  
@Cite: Bilibili -> 莫烦Python  
@Note: `多进程在不能在交互式Python运行，应写成.py文件然后执行`

In [2]:
import time

## 创建进程 & 使用队列收集返回值
* 多进程文件应使用命令行执行
* 用法和treading非常类似

In [3]:
!cat code1.py

import multiprocessing

def job(Q):
    print("new process is start\n")
    res = 0
    for i in range(10000000):
        res += i+i**2+i**3
    Q.put(res)  # * 将返回值放进队列'Q'
    print("new process is finished\n")

if __name__=='__main__':  # 多进程不加这句不行，多线程可以不加这句
    Q = multiprocessing.Queue()  # 创建队列
    process1 = multiprocessing.Process(target=job, args=(Q,))  # ! 函数有一个参数的时候，必须有逗号
    process2 = multiprocessing.Process(target=job, args=(Q,))

    process1.start()  # 进程1开始
    process2.start()  # 进程2开始

    process1.join()   # 进程1加入主进程
    process2.join()   # 进程2加入主进程

    res1 = Q.get()
    res2 = Q.get()
    print(res1, res2)
    print("主进程结束")

In [4]:
!python code1.py

new process is start

new process is start

new process is finished

new process is finished

2499999833333358333330000000 2499999833333358333330000000
主进程结束


## 时间对比：多进程能快多少？
* 对于计算密集型程序，多进程提成显著；
* 多线程实际上还是单核运算，由于IO读取限制，速度反而更慢

In [5]:
# 不使用并行

start_time = time.time()

def job():
    res = 0
    for i in range(10000000):
        res += i+i**2+i**3
    return res

res1 = job()
res2 = job()
print(res1, res2)

end_time = time.time()
print("cost time: ", end_time-start_time)

2499999833333358333330000000 2499999833333358333330000000
cost time:  9.382076025009155


In [6]:
# 使用多进程

start_time = time.time()

!python code1.py

end_time = time.time()
print("cost time: ", end_time-start_time)

new process is start

new process is start

new process is finished

new process is finished

2499999833333358333330000000 2499999833333358333330000000
主进程结束
cost time:  5.098523855209351


In [7]:
# 使用多线程

import threading
from queue import Queue

start_time = time.time()

def job(Q):
    print("new thread is start\n")
    res = 0
    for i in range(10000000):
        res += i+i**2+i**3
    Q.put(res)  # * 将返回值放进队列'Q'
    print("new thread is finished\n")

if __name__=='__main__':  # 多进程不加这句不行，多线程可以不加这句
    Q = Queue()  # 创建队列
    thread1 = threading.Thread(target=job, args=(Q,))  # ! 函数有一个参数的时候，必须有逗号
    thread2 = threading.Thread(target=job, args=(Q,))

    thread1.start()  # 进程1开始
    thread2.start()  # 进程2开始

    thread1.join()   # 进程1加入主进程
    thread2.join()   # 进程2加入主进程

    res1 = Q.get()
    res2 = Q.get()
    print(res1, res2)
    print("主线程结束")

end_time = time.time()
print("cost time: ", end_time-start_time)

new thread is start

new thread is start

new thread is finished

new thread is finished

2499999833333358333330000000 2499999833333358333330000000
主线程结束
cost time:  9.308304071426392


## 进程池Pool
* multiprocessing.Process()创建单一进程，返回值需要用Queue来承接
* multiprocessing.Pool()可创建多个进程，自动分配核心，任务函数可以有返回值

In [12]:
!cat code2.py

import multiprocessing

def job(x):
    return x*x

if __name__=='__main__':
    pool = multiprocessing.Pool()  # 创建多进程池
    results = pool.map(job, range(10))  # 将pool的值map到results中
    print(results)

In [25]:
!python code2.py

10
